## Universal Path Setup

In [ ]:
import os
import sys
import shutil
import subprocess
from pathlib import Path

# --- 1. Universal Path Setup ---
# Automatically detect if we are in Colab/Kaggle or a persistent HPC folder
CURRENT_DIR = Path.cwd()
REPO_NAME = "lip-sync-dl-f25"
REPO_URL = "https://github.com/MUKAMAFrancois/lip-sync-dl-f25.git"

print(f" Current Working Directory: {CURRENT_DIR}")

# If the repo folder doesn't exist, clone it
if not (CURRENT_DIR / REPO_NAME).exists() and not (CURRENT_DIR / ".git").exists():
    print(" Cloning repository...")
    subprocess.run(["git", "clone", REPO_URL], check=True)
    os.chdir(REPO_NAME)
elif (CURRENT_DIR / REPO_NAME).exists():
    os.chdir(REPO_NAME)

print(f" Execution Root: {Path.cwd()}")

# --- 2. Robust Dependency Installation ---
print(" Installing Dependencies...")

# Force uninstallation of incompatible numpy versions common in new environments
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"], stdout=subprocess.DEVNULL)

# Install strict requirements
# Note: --no-deps on some packages to prevent them from upgrading numpy back to 2.x
commands = [
    [sys.executable, "-m", "pip", "install", "-q", "numpy<2.0"],
    [sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"],
    [sys.executable, "-m", "pip", "install", "-q", "face-alignment", "pytorch-fid", "kagglehub", "--no-deps"],
    [sys.executable, "-m", "pip", "install", "-q", "opencv-python-headless<4.8"], # Downgrade for stability
    [sys.executable, "-m", "pip", "install", "-q", "librosa==0.10.1"]
]

for cmd in commands:
    subprocess.run(cmd, check=False)

print(" Environment Ready.")

## Setup Wav2Lip & Checkpoints

In [ ]:
# --- Setup Wav2Lip & Checkpoints ---
import os

# 1. Run the setup script to clone Wav2Lip if missing
print(" Setting up Wav2Lip Submodule...")
if not os.path.exists("setup_wav2lip.py"):
    print("!!! Error: setup_wav2lip.py not found. Are you in the right directory?")
else:
    !python setup_wav2lip.py

# 2. Verify Checkpoints
checkpoints = ["checkpoints/wav2lip_gan.pth", "checkpoints/lipsync_expert.pth"]
missing = [c for c in checkpoints if not os.path.exists(c)]

if not missing:
    print(" All checkpoints verified.")
else:
    print(f"!!! Missing checkpoints: {missing}. Retrying setup...")
    !python setup_wav2lip.py

## Download Dataset

In [ ]:
import kagglehub
import shutil
from pathlib import Path

# --- Download Dataset ---
print(" Downloading Dataset via KaggleHub...")
# Note: On PSC/HPC, ensure you have internet access or upload data manually to 'data/german'
try:
    path = kagglehub.dataset_download("francoismukama/muavic-german-sample")

    target = Path("data/german")
    target.mkdir(parents=True, exist_ok=True)

    # Source path from kagglehub download
    src_root = Path(path) / "mtedx/video/de"

    # Copy splits (Train/Val/Test)
    for split in ["train", "val", "test"]:
        src = src_root / split
        # Handle 'val' vs 'valid' naming discrepancies
        if not src.exists() and split == "val": src = src_root / "valid"

        dst = target / split
        if src.exists():
            if dst.exists(): shutil.rmtree(dst)
            shutil.copytree(src, dst)
            print(f" Data Organized: {split}")
        else:
            print(f"!!! Warning: Split '{split}' not found in source.")

except Exception as e:
    print(f"!!! Data Download Failed: {e}")
    print(" If on HPC without internet, please manually upload dataset to 'data/german'")

## preprocessing

In [ ]:
# --- Preprocessing ---
import os

# Ensure PYTHONPATH includes the current directory and Wav2Lip
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.getcwd()}/Wav2Lip"

print(" Starting Preprocessing...")
print("   (This creates 'audio.wav' and aligned face crops for training)")

!python preprocess_training_data.py \
  --data_root data/german \
  --output_root data/german/preprocessed \
  --filelist_root data/german/filelists \
  --no-zip

## Training

In [ ]:
# --- Training ---
import os

# 1. Set Path for Imports
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.getcwd()}/Wav2Lip"

# 2. Check for GPU
import torch
if not torch.cuda.is_available():
    print("!!! WARNING: No GPU detected. Training will be extremely slow or fail.")
else:
    print(f" GPU Detected: {torch.cuda.get_device_name(0)}")

# 3. Run Training
# Note: You can adjust batch_size in 'training/hparams.py' if you hit OOM errors.
print(" Starting Wav2Lip Fine-Tuning...")
!python training/train.py

## Inference

In [ ]:
# --- Full Batch Inference ---
import os
import shutil

# 1. Verify FFmpeg (Required for audio processing)
if not shutil.which("ffmpeg"):
    print("!!! FFmpeg not found! Please install it (e.g., 'apt-get install ffmpeg' or 'module load ffmpeg').")
else:
    print(" FFmpeg found.")

# 2. Set PYTHONPATH so internal imports work
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.getcwd()}/Wav2Lip"

# 3. Define Input/Output
# Run on the TEST set to evaluate performance
INPUT_DIR = "data/german/test"
OUTPUT_DIR = "results/dubbed_test"

# 4. Run the Pipeline
print(f" Starting Batch Inference on {INPUT_DIR}...")
!python pipeline/main.py \
    --input_path "{INPUT_DIR}" \
    --output_root "{OUTPUT_DIR}" \
    --source_lang "german"

## Evaluate

In [ ]:
# --- Evaluation ---
import os

os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.getcwd()}/Wav2Lip"

# Define Paths
# GT_PATH = Path to original frames (created during preprocessing)
# GEN_PATH = Path to generated video frames (you need to extract them first if evaluating video files)

# For demonstration, we assume you want to evaluate the 'test' split processed in Cell 4
# Note: To evaluate properly, you need to generate a full validation set using the model.
# This cell runs the generic evaluator script on placeholder paths.

print(" Starting Evaluation Script...")
!python evaluate.py \
    --gt_path "data/german/preprocessed/val" \
    --gen_path "dubbing_output/generated_frames_placeholder"